In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPool1D
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [21]:
df = pd.read_csv("/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv")
df.head(3)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


# Data Preprocessing

## Label Mapping

In [22]:
df['label'] = df['Category'].map({
    'ham': 0,
    'spam': 1
})
df.head(3)

,Category,Message,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1


## Data Splitting

In [23]:
X_train, X_val, y_train, y_val = train_test_split(df[['Message']], df['label'], test_size=0.2, random_state=12)

## Tokenize
use keras tokenizer to tokenize the message

In [24]:
MAX_VOCAB_SIZE = 2000

# keras tokenizer
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE) 

# train tokenizer
tokenizer.fit_on_texts(df['Message'])

# embed token into vector
sequences_train = tokenizer.texts_to_sequences(X_train['Message'])
sequences_test = tokenizer.texts_to_sequences(X_val['Message'])

In [25]:
word2idx = tokenizer.word_index

total_words = len(word2idx)
print(f' total unique words {total_words}')

 total unique words 9004


## Padding
ensure that vector sequences have the same length

In [26]:
# ensure that input sequences have the same length by padding them
data_train = pad_sequences(sequences_train)
print(data_train.shape)

# get sequence length
length = data_train.shape[1] # 175

# pad data test
data_test = pad_sequences(sequences_test, maxlen=length)

(4457, 175)


# Modeling

## RNN

In [27]:
D = 50
num_categories = 2

i = Input(shape=(length,))
x = Embedding(total_words + 1, D)(i)
x = SimpleRNN(32, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dense(num_categories)(x)

model1 = Model(i, x)

In [28]:
model1.compile(
  loss=SparseCategoricalCrossentropy(from_logits=True),
  optimizer='adam',
  metrics=['accuracy']
)

In [29]:
%%time

# training model
history = model1.fit(
  data_train,
  y_train,
  epochs=10,
  validation_data=(data_test, y_val)
)

Epoch 1/10
140/140 [==============================] - 30s 202ms/step - loss: 0.3159 - accuracy: 0.8883 - val_loss: 0.1405 - val_accuracy: 0.9623
Epoch 2/10
140/140 [==============================] - 23s 166ms/step - loss: 0.0760 - accuracy: 0.9829 - val_loss: 0.0780 - val_accuracy: 0.9794
Epoch 3/10
140/140 [==============================] - 19s 136ms/step - loss: 0.0368 - accuracy: 0.9915 - val_loss: 0.0674 - val_accuracy: 0.9794
Epoch 4/10
140/140 [==============================] - 19s 139ms/step - loss: 0.0231 - accuracy: 0.9955 - val_loss: 0.0632 - val_accuracy: 0.9812
Epoch 5/10
140/140 [==============================] - 20s 142ms/step - loss: 0.0155 - accuracy: 0.9973 - val_loss: 0.0692 - val_accuracy: 0.9803
Epoch 6/10
140/140 [==============================] - 19s 134ms/step - loss: 0.0102 - accuracy: 0.9980 - val_loss: 0.0866 - val_accuracy: 0.9794
Epoch 7/10
140/140 [==============================] - 19s 137ms/step - loss: 0.0067 - accuracy: 0.9987 - val_loss: 0.0680 - val_ac

## LSTM

In [30]:
D = 50
num_categories = 2

i = Input(shape=(length,))
x = Embedding(total_words + 1, D)(i)
x = LSTM(32, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dense(num_categories)(x)

model2 = Model(i, x)

**Code Explanation**
1. `D = 50`: This line initializes a variable D with the value 50. It's likely representing the dimensionality of the word embeddings.
2. `i = Input(shape=(length,))`: This line creates an input layer for the neural network. The input shape is specified as (length,), indicating that the model expects input sequences of a certain length.
3. `x = Embedding(total_words + 1, D)(i)`: This line adds an embedding layer to the model. The embedding layer is used to convert integer-encoded words into dense vectors of fixed size (D). The total_words + 1 is the input dimension, representing the vocabulary size.
4. `x = LSTM(32, return_sequences=True)(x)`: This line adds a Long Short-Term Memory (LSTM) layer with 32 units to the model. LSTMs are a type of recurrent neural network (RNN) designed to capture dependencies in sequential data. The return_sequences=True option indicates that the LSTM layer should return the full sequence of outputs for each input sequence.
5. `x = GlobalMaxPool1D()(x)`: This line adds a global max pooling layer to the model. Global max pooling reduces the dimensionality of the output from the LSTM layer by taking the maximum value over the sequence.
6. `x = Dense(total_categories)(x)`: This line adds a dense layer to the model with total_categories units. This layer is the output layer, and it produces the final predictions. The number of units in this layer corresponds to the number of categories or classes in the classification task.
7. `model = Model(i, x)`: This line creates a Keras Model, specifying the input (i) and output (x). The resulting model can be compiled and trained on a dataset for a specific NLP task, such as text classification.

In [31]:
model2.compile(
  loss=SparseCategoricalCrossentropy(from_logits=True),
  optimizer='adam',
  metrics=['accuracy']
)

In [32]:
%%time

# training model
history = model2.fit(
  data_train,
  y_train,
  epochs=10,
  validation_data=(data_test, y_val)
)

Epoch 1/10
140/140 [==============================] - 16s 97ms/step - loss: 0.3783 - accuracy: 0.8593 - val_loss: 0.1971 - val_accuracy: 0.9561
Epoch 2/10
140/140 [==============================] - 5s 36ms/step - loss: 0.0912 - accuracy: 0.9832 - val_loss: 0.0731 - val_accuracy: 0.9812
Epoch 3/10
140/140 [==============================] - 4s 26ms/step - loss: 0.0403 - accuracy: 0.9917 - val_loss: 0.0622 - val_accuracy: 0.9821
Epoch 4/10
140/140 [==============================] - 2s 17ms/step - loss: 0.0276 - accuracy: 0.9939 - val_loss: 0.0581 - val_accuracy: 0.9830
Epoch 5/10
140/140 [==============================] - 2s 13ms/step - loss: 0.0207 - accuracy: 0.9957 - val_loss: 0.0586 - val_accuracy: 0.9821
Epoch 6/10
140/140 [==============================] - 2s 16ms/step - loss: 0.0144 - accuracy: 0.9971 - val_loss: 0.0602 - val_accuracy: 0.9848
Epoch 7/10
140/140 [==============================] - 2s 13ms/step - loss: 0.0113 - accuracy: 0.9978 - val_loss: 0.0646 - val_accuracy: 0.981

## GRU

In [33]:
D = 50
num_categories = 2

i = Input(shape=(length,))
x = Embedding(total_words + 1, D)(i)
x = GRU(32, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dense(num_categories)(x)

model3 = Model(i, x)

In [34]:
model3.compile(
  loss=SparseCategoricalCrossentropy(from_logits=True),
  optimizer='adam',
  metrics=['accuracy']
)

In [35]:
%%time

# training model
history = model3.fit(
  data_train,
  y_train,
  epochs=10,
  validation_data=(data_test, y_val)
)

Epoch 1/10
140/140 [==============================] - 13s 81ms/step - loss: 0.3123 - accuracy: 0.8939 - val_loss: 0.0861 - val_accuracy: 0.9767
Epoch 2/10
140/140 [==============================] - 6s 44ms/step - loss: 0.0460 - accuracy: 0.9881 - val_loss: 0.0710 - val_accuracy: 0.9812
Epoch 3/10
140/140 [==============================] - 4s 26ms/step - loss: 0.0257 - accuracy: 0.9942 - val_loss: 0.0649 - val_accuracy: 0.9848
Epoch 4/10
140/140 [==============================] - 3s 25ms/step - loss: 0.0171 - accuracy: 0.9966 - val_loss: 0.0637 - val_accuracy: 0.9857
Epoch 5/10
140/140 [==============================] - 2s 15ms/step - loss: 0.0119 - accuracy: 0.9978 - val_loss: 0.0634 - val_accuracy: 0.9821
Epoch 6/10
140/140 [==============================] - 2s 15ms/step - loss: 0.0087 - accuracy: 0.9984 - val_loss: 0.0615 - val_accuracy: 0.9830
Epoch 7/10
140/140 [==============================] - 2s 11ms/step - loss: 0.0071 - accuracy: 0.9982 - val_loss: 0.0772 - val_accuracy: 0.983

## Combination

In [36]:
D = 50
num_categories = 2

i = Input(shape=(length,))
x = Embedding(total_words + 1, D)(i)
x = LSTM(16, return_sequences=True)(x)
x = GRU(32, return_sequences=True)(x)
x = GRU(64, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dense(num_categories)(x)

model4 = Model(i, x)

In [37]:
model4.compile(
  loss=SparseCategoricalCrossentropy(from_logits=True),
  optimizer='adam',
  metrics=['accuracy']
)

In [38]:
%%time

# training model
history = model4.fit(
  data_train,
  y_train,
  epochs=10,
  validation_data=(data_test, y_val)
)

Epoch 1/10
140/140 [==============================] - 21s 113ms/step - loss: 0.3710 - accuracy: 0.8737 - val_loss: 0.1112 - val_accuracy: 0.9677
Epoch 2/10
140/140 [==============================] - 5s 39ms/step - loss: 0.0608 - accuracy: 0.9845 - val_loss: 0.0790 - val_accuracy: 0.9794
Epoch 3/10
140/140 [==============================] - 4s 31ms/step - loss: 0.0323 - accuracy: 0.9919 - val_loss: 0.0676 - val_accuracy: 0.9812
Epoch 4/10
140/140 [==============================] - 4s 28ms/step - loss: 0.0210 - accuracy: 0.9942 - val_loss: 0.0631 - val_accuracy: 0.9803
Epoch 5/10
140/140 [==============================] - 4s 26ms/step - loss: 0.0126 - accuracy: 0.9966 - val_loss: 0.0787 - val_accuracy: 0.9794
Epoch 6/10
140/140 [==============================] - 4s 27ms/step - loss: 0.0089 - accuracy: 0.9973 - val_loss: 0.0842 - val_accuracy: 0.9794
Epoch 7/10
140/140 [==============================] - 4s 25ms/step - loss: 0.0064 - accuracy: 0.9987 - val_loss: 0.0909 - val_accuracy: 0.98

# Evaluation

In [39]:
preds = model1.predict(data_test)
# get the index of maximum prediction values (0/1)
preds = np.argmax(preds, axis=1)
print(preds)

print('confusion matrix')
print(confusion_matrix(y_val, preds))
print()
print('classification report')
print(classification_report(y_val, preds))

35/35 [==============================] - 1s 13ms/step
[0 0 0 ... 0 1 0]
confusion matrix
[[949   8]
 [ 12 146]]

classification report
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       957
           1       0.95      0.92      0.94       158

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [40]:
preds = model2.predict(data_test)
# get the index of maximum prediction values (0/1)
preds = np.argmax(preds, axis=1)
print(preds)

print('confusion matrix')
print(confusion_matrix(y_val, preds))
print()
print('classification report')
print(classification_report(y_val, preds))

35/35 [==============================] - 0s 3ms/step
[0 0 0 ... 0 1 0]
confusion matrix
[[954   3]
 [ 16 142]]

classification report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       957
           1       0.98      0.90      0.94       158

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [41]:
preds = model3.predict(data_test)
# get the index of maximum prediction values (0/1)
preds = np.argmax(preds, axis=1)
print(preds)

print('confusion matrix')
print(confusion_matrix(y_val, preds))
print()
print('classification report')
print(classification_report(y_val, preds))

35/35 [==============================] - 0s 3ms/step
[0 0 0 ... 0 1 0]
confusion matrix
[[954   3]
 [ 16 142]]

classification report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       957
           1       0.98      0.90      0.94       158

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [42]:
preds = model4.predict(data_test)
# get the index of maximum prediction values (0/1)
preds = np.argmax(preds, axis=1)
print(preds)

print('confusion matrix')
print(confusion_matrix(y_val, preds))
print()
print('classification report')
print(classification_report(y_val, preds))

35/35 [==============================] - 2s 9ms/step
[0 1 0 ... 0 1 0]
confusion matrix
[[946  11]
 [ 11 147]]

classification report
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       957
           1       0.93      0.93      0.93       158

    accuracy                           0.98      1115
   macro avg       0.96      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115

